In [15]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import os

In [3]:
data=pd.read_csv("fully_processed_data.csv")
data.head(2)

,Unnamed: 0,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,project_title,price,std_price,nrm_price
0,0,in,mrs,grades_prek_2,0,0,literacy_language,esl_literacy,students english learners working english seco...,educational support english learners home,154.6,-0.390533,0.015397
1,1,fl,mr,grades_6_8,7,1,history_civics_health_sports,civics_government_teamsports,students arrive school eager learn polite gene...,wanted projector hungry learners,299.0,0.002396,0.029839


In [4]:
docs=list(df.essay.values)
labels=np.array(df.project_is_approved.values)

preprocessed_essays = data['essay'].values


In [4]:
"""# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better and good.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])"""

"# define documents\ndocs = ['Well done!',\n        'Good work',\n        'Great effort',\n        'nice work',\n        'Excellent!',\n        'Weak',\n        'Poor effort!',\n        'not good',\n        'poor work',\n        'Could have done better and good.']\n# define class labels\nlabels = array([1,1,1,1,1,0,0,0,0,0])"

In [5]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
print(vocab_size)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
max_len=0
all_lengths=[]
for sent in docs:
    length=len(sent.split())
    all_lengths.append(length)
print(max(all_lengths))

320


In [7]:
# pad documents to a max length of 4 words
max_length = max(all_lengths)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[   1   82   48 ...    0    0    0]
 [   1 1360    2 ...    0    0    0]
 [ 788 5837    5 ...    0    0    0]
 ...
 [  79   51  745 ...    0    0    0]
 [   1  175   72 ...    0    0    0]
 [ 326   28   83 ...    0    0    0]]


In [8]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [13]:
print(len(embedding_matrix))
print(len(embedding_matrix[0]))

56244
300


In [14]:
# stronging variables into pickle files python: http://www.jessicayung.com/how-to-use-pickle-to-save-and-load-variables-in-python/
# make sure you have the glove_vectors file
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())
    
# average Word2Vec
# compute average word2vec for each review.
avg_w2v_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors.append(vector)

print(len(avg_w2v_vectors))
print(len(avg_w2v_vectors[0]))

NameError: name 'pickle' is not defined

In [9]:
# define model
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 320, 300)          16873200  
_________________________________________________________________
flatten_1 (Flatten)          (None, 96000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 96001     
Total params: 16,969,201
Trainable params: 96,001
Non-trainable params: 16,873,200
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Accuracy: 92.885911
